<a href="https://colab.research.google.com/github/layafakher/Resume_Information_Extraction/blob/main/Resume_Information_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports


In [ ]:
!pip install pypdf2

In [ ]:
from PyPDF2 import PdfReader, PdfFileWriter, PdfFileMerger  # Library for working with PDF files
import os  # Library for interacting with the operating system
import pandas as pd  # Library for working with data in tabular form
import re  # Library for regular expressions
from nltk.corpus import stopwords  # Library for stopwords
import re  # Library for regular expressions
import nltk  # Natural Language Processing library
from nltk import ne_chunk, pos_tag, word_tokenize  # Natural Language Processing functions
from nltk.tree import Tree  # nltk tree structure
import spacy  # Natural Language Processing library
nltk.download('stopwords')  # Download necessary data from the nltk server
from nltk.corpus import stopwords  # Library for stopwords
nltk.download('punkt')  # Download necessary data from the nltk server
nltk.download('averaged_perceptron_tagger')  # Download necessary data from the nltk server
nltk.download('maxent_ne_chunker')  # Download necessary data from the nltk server
nltk.download('words')  # Download necessary data from the nltk server
import spacy  # Natural Language Processing library
import en_core_web_sm  # Language model for spacy
from spacy.matcher import Matcher  # Matcher for spacy
import pandas as pd  # Library for working with data in tabular form
import spacy  # Natural Language Processing library
from nltk import ne_chunk, pos_tag, word_tokenize  # Natural Language Processing functions
from nltk.tree import Tree  # nltk tree structure
nlp = spacy.load('en_core_web_sm')  # Load spacy language model
from collections import defaultdict  # Library for default dictionaries
from nltk.corpus import stopwords  # Library for stopwords
from nltk.tokenize import word_tokenize  # Functions for tokenizing sentences


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
SST_HOME='drive/My Drive/Sample Data Resume/'

#Converting PDFs to text

In [ ]:
def pdftotext(path):
    """
    Function to convert a PDF file to text.

    :param path: Path of the PDF file
    :return: Extracted text from the PDF
    """
    pdfFileObj = open(path, 'rb')
    pdfFileReader = PdfReader(pdfFileObj)
    num_pages = len(pdfFileReader.pages)
    currentPageNumber = 0
    text = ''
    while(currentPageNumber < num_pages ):
        pdfPage = pdfFileReader.pages[num_pages-1]
        text = text + pdfPage.extract_text()
        text = text.encode('utf-8', 'ignore').decode('utf-8')
        currentPageNumber += 1

    return (text)

In [ ]:
PDFs = []
for filename in os.listdir(SST_HOME):
    f = os.path.join(SST_HOME, filename)
    # checking if it is a file
    if os.path.isfile(f):
        PDFs.append(f)
PDFs

['drive/My Drive/Sample Data Resume/CV-LayaFakher.pdf',
 'drive/My Drive/Sample Data Resume/10985403.pdf',
 'drive/My Drive/Sample Data Resume/11890896.pdf',
 'drive/My Drive/Sample Data Resume/12377803.pdf',
 'drive/My Drive/Sample Data Resume/12230301.pdf',
 'drive/My Drive/Sample Data Resume/11704150.pdf',
 'drive/My Drive/Sample Data Resume/11653906.pdf']

#Extracting Information

In [ ]:
nlp = en_core_web_sm.load()

In [ ]:
def match_name(name):
    # Loading the English language model for Spacy
    nlp = spacy.load('en_core_web_sm')
    matcher = Matcher(nlp.vocab)

    patterns = [
        [{'POS': 'PROPN'}, {'POS': 'PROPN'}],  # First name and Last name
        [{'POS': 'PROPN'}, {'POS': 'PROPN'}, {'POS': 'PROPN'}],  # First name, Middle name, and Last name
        [{'POS': 'PROPN'}, {'POS': 'PROPN'}, {'POS': 'PROPN'}, {'POS': 'PROPN'}]  # First name, Middle name, Middle name, and Last name
    ]

    for pattern in patterns:
        matcher.add('NAME', patterns=[pattern])

    doc = nlp(name)
    matches = matcher(doc)

    for match_id, start, end in matches:
        return True

    return False

In [ ]:
def extract_names(text):
  names = []
  spacy_parser = nlp(text)
  for entity in spacy_parser.ents:
      if entity.label_ ==  "PERSON" and match_name(entity.text):
          return names

In [ ]:
def extract_email_from_resume(string):
  # Regular expression for extracting email addresses
    r = re.compile(r'[\w\.-]+@[\w\.-]+')
    return r.findall(string)


In [ ]:
def extract_skills(text):
    skills = defaultdict(int)
    stop_words = stopwords.words('english')
    words = word_tokenize(text)
    words = [word.lower() for word in words if word.isalpha()]
    for word in words:
        is_skill = True
        spacy_parser = nlp(word)
        for entity in spacy_parser.ents:
            if entity.label_  in ["DATE", "GPE", "LOC", "PRODUCT", "EVENT", "GPE", "ORG", "FAC" , "NORP"]:
                is_skill = False
        if word not in stop_words and is_skill:
            skills[word] += 1

    # Extracting top 20 skills
    top_skills = sorted(skills, key=skills.get, reverse=True)[:20]
    return top_skills


#Resume Information Extraction

In [ ]:
def resume_information_extractor(resume_paths):
    for resume_path in resume_paths:
        text = pdftotext(resume_path)

        print("Resume:", resume_path)

        name = extract_names(text)
        if name:
            print("Name:", name)
        else:
            print("Name not found")


        email = extract_email_from_resume(text)
        if email:
            print("Email:", email)
        else:
            print("Email not found")

        extracted_skills = extract_skills(text)
        if extracted_skills:
            print("Skills:", extracted_skills)
        else:
            print("No skills found")

        print()


In [ ]:
resume_information_extractor(PDFs)